# Potential Data Creation

Given a periodic potential $v$, symbolically calculate significant data, like characteristic polynomials etc

We work with the monodromy matrix, as it is easier to calculate. Cf. Remark 3.9

In particular, $M_{1,1} = \pm \chi_{A_{0..K-1}}(E)$ and $M_{1,2} = \pm \chi_{A_{0..K-2}}(E)$.

In [1]:
# Run this cell once at startup

l = var('l')
E = var('E')


def mon(v):
#
# Calculate the general monodromy matrix with scaling l #
# and energy E
#
# Example:
# v = [1,1,0]; mon(v).expand()
# > [E^2 - E*l - 1            -E]
#   [        E - l            -1]
#
##########################
    M = identity_matrix(2)
    for vv in v:
        M = matrix(2,2,[E - l*vv, -1, 1, 0])*M
    return M

def lim_pots(v):
#
# determine all unique limit potentials of a given
# periodic potential v by considering all shifts and flips
#
##########################
    v_rev = deepcopy(v)
    lim = shift_pots(v)
    v_rev.reverse()
    lim_rev = shift_pots(v_rev)
    for pot in lim_rev:
        if pot not in lim:
            lim.append(pot)
    return lim
    
def shift_pots(v):
#
# determine all unique shifts of a given periodic potential v
#
##########################
    shift = []
    for c in range(len(v)):
        if v not in shift:
            shift.append(v)
        v = right_shift(v)
    return shift
    
def right_shift(a):
    return [a[-1] , *a[:-1]]

def cutoff(v,K):
#
# (deprecated, everything can also be done with `mon`)
#
# determine the cutoff matrix H_{0..K} with potential v[0:K-1]
#
##########################
    A = matrix(SR,K+1,K+1) #start enumerating from 0
    A = A + matrix.toeplitz([0,1] + [0]*(K-1), [1]+[0]*(K-1))
    for k in range(K+1):
        A[k,k] = v[k]
    return A

# Consider E=0

Given period length, systematically check for every monodromy matrix if there exists a scaling factor $l$ such that $M_{2,1} = 0$. If such scalings exist, check whether the full operator is also not invertible by verifying the trace condition.

In [2]:
# this cell contains basic routines that need to be loaded 
# before data generation can start

def analyse_v(v):
##########################
# calculate monodromy, zeros and check trace. 
# return a dictionary containing symbolic elements
# 
# This routine only analyses the energy E = 0.
#
# example:
# v = [1,1,0]; analyse_v(v)
##########################
    data = {}
#monodromy
    M = mon(v).expand() 
    data['mon'] = M
#zeros
    sols = solve(M[1][0].subs(E=0) == 0, l)
    sol = list(map(lambda s: s.rhs(), sols))
    sol.sort()
    data['sol'] = sol
#trace
    data['trace'] = list(map(lambda s: M.subs(E=0).trace().subs(l=s), sol))
    return data

# Data Generation

Run the cell below with `Kmin = 0` and `Kmax = 9` in order to generate all data for Example 3.17. 

This will produce a dictionary `pots` where keys are given by stringified potentials and values consist of the symbolic monodromy matrix, the zeros of the matrix entry and the value of the trace.

## Example Usage of Data

In order to access the values in the dictionary for the potential `v = [0,1,1]` use
```python
v = [0,1,1]
pots[str(v)]
```

## Exporting the Results

It is possible to export the files in JSON, see below.


In [3]:
# input
Kmin = 1 
Kmax = 9 #maximal period length, not larger than 9
sigma = [0,1] # alphabet (only tested with sigma = [0,1])0

pots = {} #empty dictionary
for K in range(Kmin,Kmax + 1):
    v_list = Tuples(sigma,K).list() 
    for v in v_list:
        pots[str(v)] = deepcopy(analyse_v(v))

# Result Filtering

The dictionary `pots` contains all possible periodic potentials. In order to filter the results we need to do 3 things

1. create a suitable list of potentials
2. decide whether we want to filter out candidates that don't fulfill the trace condition
3. decide whether we want to allow irrational scaling factors

## Interpretation of Results

### Example 1
```python
v_list = Tuples(sigma,2).list()
for v in v_list:
    print([v, [s for s in pots[str(v)]['sol']],  [s.n() for s in pots[str(v)]['trace']]])
```
gives as first line of output
```
[[0, 0], [r1], [-2.00000000000000]]
```
This means the entry $M_{2,1}$ of monodromy matrix for the potential $v = (0,0)$ is zero for all scalings (`r1` corresponds to $\mathbb{R}$) and that the value of the trace of $M$ is always $-2$.
The line
```
[[1, 0], [0], [-2.00000000000000]]
```
means that the entry $M_{2,1}$ of monodromy matrix for the potential $v = (1,0)$ is zero for the scaling `l = 1` and that the value of the trace of $M$ is, in this case, $-2$.

### Example 2

Similarly to Example 1, we can look at potentials with period $K =  5$. A line like
```
[[1, 0, 1, 1, 0], [-1/2*sqrt(2), 1/2*sqrt(2)], [2.12132034355964, -2.12132034355964]]
```
means that the entry $M_{2,1}$ of monodromy matrix for the potential $v = (1, 0, 1, 1, 0)$ is zero for the tuple of $\left( -\frac{\sqrt{2}}{2}, \frac{\sqrt{2}}{2}\right)$ and that the corresponding values of the trace of $M$ are $(2.12\dots, -2.12\dots)$.

In [4]:
v_list = Tuples(sigma,5).list()

for v in v_list:
    if true in [abs(t).n() > 2 for t in pots[str(v)]['trace']]: #uncomment this line to check the trace condition
        if true in [s in QQ for s in pots[str(v)]['sol']]:      #uncomment this line to check for rational roots
            print([v, [s for s in pots[str(v)]['sol']],  [s.n() for s in pots[str(v)]['trace']]])

# JSON Output

Export the `pots` dictionary in .json file format (and back).

You can read the input using a symple python program
```python
#!/bin/python3

import json

with open('pots.json') as json_file:
    data = json.load(json_file)
    for key in data:
        print(key)
```

or an online json reader like https://jsonformatter.org/json-reader

In [5]:
# run this cell to load json output routines

import json
def jsonify_dict(input_dict):
##########################
# create json object from dictionary, 
# stringify everything for serialization
##########################
    json_object ={}
    for key in input_dict:
        dictionary = {}
        dictionary['mon'] = str(input_dict[key]['mon'])
        dictionary['sol'] = str(input_dict[key]['sol'])
        dictionary['trace'] = str(input_dict[key]['trace'])
        json_object[str(key)] = deepcopy(dictionary)
    return json_object

def read_json_data(json_object):
##########################
# do the opposite of jsonify_data
#
# todo
##########################
    return None

In [6]:
# run this cell in order to save pots into json file

json_dict = jsonify_dict(pots)

with open('pots.json', 'w') as outfile:
    json.dump(json_dict,outfile)